In [1]:
import pandas as pd
import numpy as np

from src.PortfolioIndicator import PortfolioIndicator
from src.PortfolioConstructor import PortfolioConstructor

In [2]:
data_directory = "data/"
file = "us_stock_price_2015_2019.csv"

In [108]:
df = pd.read_csv(data_directory+file)
df = df.iloc[:,1:]
df["date"] = pd.to_datetime(df["date"])
df = df[["date","TICKER","PRC"]]

In [109]:
price_df = df

In [111]:
price_df_wide = price_df.pivot_table(values="PRC",columns="TICKER",index="date")

In [112]:
price_df_wide = price_df_wide.fillna(method="ffill")

In [113]:
universe_df = price_df_wide

In [114]:
sub_universe_df = universe_df[universe_df.index>"2018-01-01"].dropna(axis=1)

In [115]:
universe_df = sub_universe_df

In [116]:
momentum_period = 3
waiting_time = 1

In [117]:
complete_ret_sub_df = universe_df.pct_change(periods=momentum_period).shift(waiting_time)

In [118]:
complete_ret_sub_df_long = complete_ret_sub_df.stack()

In [119]:
complete_ret_sub_df_long

date        TICKER
2018-01-08  A         0.034024
            AA       -0.019576
            AAAP      0.000244
            AABA      0.045946
            AAC      -0.016181
                        ...   
2019-12-31  ZU        0.000000
            ZUMZ      0.052371
            ZX        0.000000
            ZYME     -0.003333
            ZYNE     -0.060000
Length: 4463554, dtype: float64

In [120]:
signal_df = complete_ret_sub_df_long.index.to_frame()
signal_df.index = range(0,signal_df.shape[0])
signal_df["momentum"] = list(complete_ret_sub_df_long)
signal_df.columns=["date","ticker","momentum"]

In [121]:
rebalance_date_series = list(complete_ret_sub_df.index)

In [122]:
portfolio_indicator = PortfolioIndicator()

In [123]:
portfolio_indicator = portfolio_indicator.create_indicator(signal_df,"momentum",rebalance_date_series,5)

In [124]:
complete_sub_df_1 = universe_df.pct_change()
complete_sub_df_1_long = complete_sub_df_1.stack()

In [125]:
index_df = complete_sub_df_1_long.index.to_frame()
index_df["ret"] = complete_sub_df_1_long
index_df.index=range(0,index_df.shape[0])
index_df.columns = ["date","ticker","ret"]

In [126]:
return_df = index_df

In [127]:
portfolio_indicator

,date,ticker,momentum_indicator
0,2018-01-08,A,4
1,2018-01-08,AA,0
2,2018-01-08,AAAP,2
3,2018-01-08,AABA,4
4,2018-01-08,AAC,0
...,...,...,...
4463549,2019-12-31,ZU,2
4463550,2019-12-31,ZUMZ,4
4463551,2019-12-31,ZX,2
4463552,2019-12-31,ZYME,1


In [128]:
merge_df = return_df.merge(portfolio_indicator,on=["date","ticker"])

In [129]:
portfolio_constructor = PortfolioConstructor(merge_df)

In [130]:
strategy_ret_df = portfolio_constructor.create_portfolio_series("momentum_indicator",equal_weight=True)

In [131]:
strategy_ret_df = strategy_ret_df.replace([np.inf, -np.inf], np.nan)

In [132]:
strategy_ret_df = strategy_ret_df.dropna()

In [133]:
strategy_cum_ret_df = (strategy_ret_df+1).cumprod()

In [134]:
strategy_ret_df.describe()

momentum_indicator,0,1,2,3,4
count,352.000000,352.000000,352.000000,352.000000,352.000000
mean,-0.238450,-0.018604,-0.055199,-0.029094,-0.014096
std,3.997285,0.022958,0.521586,0.016897,0.908828
min,-46.936462,-0.207521,-9.800446,-0.106858,-1.977226
25%,-0.275501,-0.027407,-0.039631,-0.039205,-0.036012
50%,-0.123632,-0.015356,-0.021712,-0.026908,-0.017563
75%,-0.058122,-0.006605,-0.009328,-0.017421,-0.006393
max,16.994954,0.203949,0.024468,0.017170,16.533791
